In [14]:
from transformers import BertTokenizerFast, BertForPreTraining,BertForTokenClassification ,Trainer, TrainingArguments
import os
import torch 
import pandas as pd
import datasets

In [15]:
torch.cuda.is_available()

True

In [16]:
from array import array
import numpy as np
def loadData():
    list_subfolders_with_paths = [f.path for f in os.scandir("./annotation_sentence") if f.is_dir()]
    data = {"texts": [],"labels":[]}
    for i in list_subfolders_with_paths:
    #print("new File")
        texts=[]
        labels=[]
        with open(f"{i}/admin.conll","r", encoding="utf-8-sig") as file: 
            for j in  file.readlines():
                if(j.strip() == ""):
                    continue
                sp = j.split(" ")
                if("null" in sp[1].rstrip("\n")):
                    print(i)
                texts.append(sp[0])
                labels.append(sp[1].rstrip("\n"))
                #print(f"Text: {sp[0]}")
                #print(f"Label: {sp[1]}")
            data["texts"].append((texts))
            data["labels"].append(( labels))
    
    
    return data

In [17]:
from datasets import Dataset , ClassLabel ,Features ,Sequence ,Value

In [18]:
traindata = loadData()
texts = traindata["texts"]
tags = traindata["labels"]

In [19]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [20]:
list(unique_tags)

['I-SENTENCE', 'O', 'B-SENTENCE']

In [21]:
dataset = Dataset.from_dict(loadData(),features=Features( {"texts": Sequence(Value("string")),'labels':Sequence(ClassLabel(names=list(unique_tags),num_classes =len(unique_tags)))}))

In [22]:
(dataset.features["labels"].feature)._str2int 
#{str(v): k for k, v in (dataset.features["labels"].feature)._str2int.items()}

{'I-SENTENCE': 0, 'O': 1, 'B-SENTENCE': 2}

In [23]:
from collections import Counter
x = [i if(len(i) > 1) else i for i in list(sum(tags, []))]
di = dict(Counter(x))
z = {}

for i in di.keys():

    z[i] = round ((di[i]/(len(x)))*100,2)
    

z

{'O': 9.13, 'B-SENTENCE': 6.05, 'I-SENTENCE': 84.82}

In [24]:
dataset = dataset.train_test_split(test_size=0.2)

In [25]:
(dataset["train"].features["labels"].feature)

ClassLabel(num_classes=3, names=['I-SENTENCE', 'O', 'B-SENTENCE'], id=None)

In [26]:
dataset["train"][0]

{'texts': ['BETRUG',
  'MIT',
  'POLIZISTEN-TRICK',
  'Bankerin',
  'rettet',
  'für',
  'Seniorin',
  '12',
  '000',
  'Euro',
  'Die',
  'Seniorin',
  'wollte',
  '12.000',
  'Euro',
  'abheben',
  '-',
  'eine',
  'Bankangestellte',
  'verhinderte',
  'das',
  '(',
  'Symbolbild',
  ')',
  'Foto',
  ':',
  'picture',
  'alliance',
  '/',
  'VisualEyze',
  '18.08.2021',
  '-',
  '15',
  ':',
  '20',
  'Uhr',
  'Kassel',
  '–',
  'Zum',
  'Glück',
  'hat',
  'hier',
  'eine',
  'Bankangestellte',
  'aufgepasst',
  '!',
  'Ihr',
  'beherztes',
  'Eingreifen',
  'hat',
  'verhindert',
  ',',
  'dass',
  'eine',
  'Seniorin',
  '(',
  '79',
  ')',
  'um',
  '12',
  '000',
  'Euro',
  'betrogen',
  'wurde',
  '.',
  'Der',
  'Angestellten',
  'kam',
  'es',
  'komisch',
  'vor',
  ',',
  'dass',
  'die',
  'Seniorin',
  'am',
  'Dienstagmittag',
  'einen',
  'so',
  'hohen',
  'Geldbetrag',
  'abheben',
  'wollte',
  '.',
  'Als',
  'die',
  'aufmerksame',
  'Mitarbeiterin',
  'der',
  'K

In [27]:
label_names = dataset["train"].features[f"labels"].feature.names

In [28]:
len(label_names)

3

In [29]:
#Load Model and Tokenizer
from transformers import AutoTokenizer

tokenizer = BertTokenizerFast.from_pretrained('deepset/gbert-large')
model = BertForTokenClassification.from_pretrained('deepset/gbert-large', num_labels=len(label_names) )

Some weights of the model checkpoint at deepset/gbert-large were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from t

In [30]:
model.config.label2id =  (dataset["train"].features["labels"].feature)._str2int 
model.config.id2label =  {str(v): k for k, v in (dataset["train"].features["labels"].feature)._str2int.items()}

In [31]:
def tokenize_and_align_labels(examples):
    #print(examples.keys())
    tokenized_inputs = tokenizer(examples["texts"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

In [32]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [33]:
def split(examples):
    dic ={"input_ids":[],"token_type_ids":[],"attention_mask":[],"labels":[]}
    for j in examples:
        if j == "texts":
            continue
        for i in range(len(examples["input_ids"])):
            arr = np.array_split(examples[j][i],2)
            dic[j].append(arr[0])
            dic[j].append(arr[1])
    return dic

In [34]:
test = tokenized_datasets.map(split,remove_columns=["input_ids","token_type_ids","attention_mask","labels","texts"], batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [35]:
full_train_dataset = test["train"]
full_eval_dataset = test["test"]

In [36]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [37]:
from datasets import load_metric
import numpy as np
metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

In [38]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [39]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=100,
    weight_decay=0.01,
)

In [40]:
trainer = Trainer(model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset,compute_metrics=compute_metrics, data_collator=data_collator,
    tokenizer=tokenizer)

In [41]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 78
  Num Epochs = 100
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 1300


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Sentence F1
1,No log,0.547122,0.000000,0.000000,0.000000,0.845537,0.000000
2,No log,0.527561,0.000000,0.000000,0.000000,0.845537,0.000000
3,No log,0.433855,0.000000,0.000000,0.000000,0.845537,0.000000
4,No log,0.383919,0.067797,0.018433,0.028986,0.903841,0.028986
5,No log,0.141118,0.350649,0.248848,0.291105,0.946947,0.291105
6,No log,0.096307,0.579545,0.705069,0.636175,0.967671,0.636175
7,No log,0.095735,0.613757,0.534562,0.571429,0.963526,0.571429
8,No log,0.065129,0.757848,0.778802,0.768182,0.978447,0.768182
9,No log,0.075462,0.719512,0.815668,0.764579,0.976789,0.764579
10,No log,0.066883,0.772321,0.797235,0.784580,0.979000,0.784580


***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 

TrainOutput(global_step=1300, training_loss=0.029802646596844378, metrics={'train_runtime': 353.991, 'train_samples_per_second': 22.034, 'train_steps_per_second': 3.672, 'total_flos': 3596883492492180.0, 'train_loss': 0.029802646596844378, 'epoch': 100.0})

In [42]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 20
  Batch size = 6


{'eval_loss': 0.12236879020929337,
 'eval_overall_precision': 0.8340807174887892,
 'eval_overall_recall': 0.8571428571428571,
 'eval_overall_f1': 0.8454545454545455,
 'eval_overall_accuracy': 0.9823155567836419,
 'eval_SENTENCE_f1': 0.8454545454545455,
 'eval_runtime': 0.2854,
 'eval_samples_per_second': 70.084,
 'eval_steps_per_second': 14.017,
 'epoch': 100.0}

In [43]:
model.save_pretrained('./NEWS_Sentence_2/')

Configuration saved in ./NEWS_Sentence_2/config.json
Model weights saved in ./NEWS_Sentence_2/pytorch_model.bin


In [32]:
tokenizer.save_pretrained('./NEWS_TOK_2/')

tokenizer config file saved in ./NEWS_TOK_2/tokenizer_config.json
Special tokens file saved in ./NEWS_TOK_2/special_tokens_map.json


('./NEWS_TOK_2/tokenizer_config.json',
 './NEWS_TOK_2/special_tokens_map.json',
 './NEWS_TOK_2/vocab.txt',
 './NEWS_TOK_2/added_tokens.json',
 './NEWS_TOK_2/tokenizer.json')